In [67]:
import os
import logging
import pandas as pd
import mysql.connector

In [68]:
MYSQL_USERNAME=os.environ['MYSQL_USERNAME']
MYSQL_PASSWORD=os.environ['MYSQL_PASSWORD']
MYSQL_DATABASE=os.environ['MYSQL_DATABASE']
MYSQL_HOST=os.environ['MYSQL_HOST']

try:
    # Establish connection
    conn = mysql.connector.connect(
        host=MYSQL_HOST,  # Change to your MySQL server
        user=MYSQL_USERNAME,
        password=MYSQL_PASSWORD,
        database=MYSQL_DATABASE  # Optional: specify database
    )

    if conn.is_connected():
        print("Connected to MySQL successfully!")

except mysql.connector.Error as e:
    print("Error:", e)

Connected to MySQL successfully!


In [69]:
# Create a cursor object
cursor = conn.cursor()

In [70]:
def extract(cursor, query):
    # Execute a query
    cursor.execute(query)
    data = cursor.fetchall()
    
    # Get column names (metadata)
    columns = [desc[0] for desc in cursor.description]

    # Convert the data into a pandas DataFrame
    df = pd.DataFrame(data, columns=columns)
    
    return df

In [77]:
query="""
SELECT p.ProductID, p.ProductName, pc.CategoryName, sc.SubCategoryName, p.Price, p.ProductionDate, p.ExpirationDate
FROM Product p
JOIN ProductCategory pc
ON p.CategoryID = pc.CategoryID
JOIN ProductSubcategory sc
ON p.SubcategoryID = sc.SubcategoryID;
"""
products_df = extract(cursor, query)
products_df

,ProductID,ProductName,CategoryName,SubCategoryName,Price,ProductionDate,ExpirationDate
0,1,Samsung Galaxy S21,Electronics,Smartphones,799.99,2021-01-05,None
1,2,Dell XPS 13,Electronics,Laptops,999.99,2020-10-10,None
2,3,Levi's 501 Jeans,Clothing,Men's Apparel,59.99,2021-07-01,None
3,4,Nike Women's Running Shoes,Clothing,Women's Apparel,89.99,2022-06-15,None
4,5,Oster Blender,Home Appliances,Kitchen Appliances,49.99,2020-09-20,None
5,6,Living Room Sofa,Furniture,Home Decor,499.99,2021-12-01,None
6,7,The Great Gatsby,Books,Fiction,15.99,2020-04-10,2024-04-10
7,8,The Lean Startup,Books,Non-Fiction,20.99,2020-01-10,2025-01-10
8,9,Uno Card Game,Toys,Board Games,19.99,2019-10-01,None
9,10,Camping Tent,Sporting Goods,Outdoor Equipment,120.00,2021-05-01,None


In [78]:
# Close the connection
cursor.close()
conn.close()
print("Connection closed.")

Connection closed.


In [79]:
import psycopg2
from psycopg2 import sql

In [80]:
PG_USERNAME=os.environ['PG_USERNAME']
PG_PASSWORD=os.environ['PG_PASSWORD']
PG_DATABASE=os.environ['PG_DATABASE']
PG_HOST=os.environ['PG_HOST']

print(PG_HOST, PG_USERNAME, PG_PASSWORD, PG_DATABASE)

172.18.0.2 biruni P@ssw0rd walmart_dwh


In [81]:
# Database connection parameters
db_config = {
    'dbname': PG_DATABASE,
    'user': PG_USERNAME,
    'password': PG_PASSWORD,
    'host': PG_HOST,  # or your database host
    'port': 5432          # default PostgreSQL port
}

In [82]:
# Connect to the database
try:
    conn = psycopg2.connect(**db_config)
    print("Connected to the database!")
except Exception as e:
    print(f"Unable to connect to the database: {e}")


Connected to the database!


In [83]:
# Create a cursor object to interact with the database
cursor = conn.cursor()

In [84]:
query = sql.SQL(f"""
    select * from dimproduct;
""")

In [85]:
cursor.execute(query)
columns = [desc[0] for desc in cursor.description]
columns

['productsk',
 'productbk',
 'productname',
 'categoryname',
 'subcategoryname',
 'price',
 'productiondate',
 'expirationdate']

In [86]:
query = sql.SQL(f"""
    INSERT INTO dimproduct (productbk, productname, categoryname, subcategoryname, price, productiondate, expirationdate)
    VALUES (%s, %s, %s, %s, %s, %s, %s);
""")

In [87]:
def load(cursor, query, data):
    for i, row in data.iterrows():

        try:
            cursor.execute(query, tuple(row))
        except Exception as e:
            conn.rollback()  # Rollback in case of error
            print(f"Error loading product: {e}")

    cursor.execute('commit')
    print("Data Loaded successfully!")

In [88]:
load(cursor, query, products_df)

Data Loaded successfully!


In [89]:
# Close the cursor and connection
cursor.close()
conn.close()
print("Connection closed.")

Connection closed.
